In [6]:
import cv2
from PIL import Image
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
import sys

from fr_utils import *
from faceDetector import *
from inception_blocks_v2 import *
from parameters import *

from keras.models import Sequential,Model
from keras.layers import Conv2D,ZeroPadding2D,Activation,Input,concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D,AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda,Flatten,Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')

%matplotlib inline

In [7]:
'''def load_model():
    model = faceRecoModel(input_shape = (3,96,96))
    model.compile(optimizer= 'adam',loss= triplet_loss,metrics = ['accuracy'])
    load_weights_from_FaceNet(model)
    print("Model loaded")
    return model
'''

def triplet_loss(y_true, y_pred, alpha = ALPHA):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))    
    return loss

'''def add_face(name):
    os.mkdir("images/" + str(name))
    cam = cv2.VideoCapture(0)

    #we'll use the 'haarcascade_frontalface_default.xml' which is in the same folder as this code
    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    # For each person, enter one numeric face id. for now this will be used to make unique ID

    print("\n [INFO] Initializing face capture. Look the camera and wait ...")
    # Initialize individual sampling face count
    count = 0

    #start detect your face and take 30 pictures
    while(True):

        ret, img = cam.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 1.3, 5)

        for (x,y,w,h) in faces:

            cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)
            count += 1

            # Save the captured image into the datasets folder
            cv2.imwrite("images/" + str(name) +"/" + str(name) + '_' + str(count) + ".jpg", gray[y:y+h,x:x+w])

            cv2.imshow('face', img)

        k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            break
        elif count >= 5: # Take 5 face sample and stop video
             break

    # Do a bit of cleanup
    print("\n [INFO] Exiting Program and cleanup stuff")
    cam.release()
    cv2.destroyAllWindows()'''
    
def add_face_database(image_path,name,model):
    database[name] = img_to_encoding(image_path,model)
    return str(name) + " added"

def verify(image_path, identity, database, model):
    
    encoding = img_to_encoding(image_path, model, False)
    min_dist = 1000
    for  pic in database:
        dist = np.linalg.norm(encoding - pic)
        if dist < min_dist:
            min_dist = dist
    print(identity + ' : ' +str(min_dist)+ ' ' + str(len(database)))
    
    if min_dist<THRESHOLD:
        door_open = True
    else:
        door_open = False
        
    return min_dist, door_open

'''def recognize_face(image_path,database,model):
    encoding = img_to_encoding(image_path,model)
    min_dist = 100
    for(name , db_encoding) in database.items():
        dist = np.linalg.norm(encoding - database[name])
        if dist<min_dist:
            min_dist = dist
            identity = name
    if min_dist > 0.7:
        print("Intruder,distance is " + str(min_dist))
    else:
        print("Welcome " +str(identity) +",distance is " + str(min_dist))
    return min_dist,identity

def img_to_encoding(image_path, model):
    img1 = cv2.imread(image_path, 1)
    img = img1[...,::-1]

    classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(gray, 1.3, 5)
    print("Found {0} face(s)!".format(len(faces)))
    plt.figure(1)
    for (x, y, w, h) in faces:
        cv2.rectangle(img1, (x-10, y-10), (x+w+10, y+h+10), (0, 255, 0), 2)
        img=img[y-10:y+h+10,x-10:x+w+10]
        face_detected = Image.fromarray(img1,'RGB')
        plt.subplot(121)
        plt.title("Face detected image")
        plt.imshow(np.asarray(face_detected))

    crp_image = Image.fromarray(img,'RGB')
    plt.subplot(122)
    plt.title("Cropped image")
    plt.imshow(np.asarray(crp_image))
    plt.show()

    img = cv2.resize(img,(96,96))
    img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)
    x_train = np.array([img])
    #embedding = model.predict_on_batch(x_train)
    #return embedding

    def show_image(true_image,test_image):
        img1 = cv2.imread(true_image)
        b,g,r = cv2.split(img1)
        img1 = cv2.merge([r,g,b])
        plt.figure(1)
        plt.subplot(121)
        plt.title("True Image")
        plt.imshow(img1)
        img2 = cv2.imread(test_image)
        b,g,r = cv2.split(img2)
        img2 = cv2.merge([r,g,b])
        plt.subplot(122)
        plt.title("Test Image")
        plt.imshow(img2)
        plt.show()'''

'def recognize_face(image_path,database,model):\n    encoding = img_to_encoding(image_path,model)\n    min_dist = 100\n    for(name , db_encoding) in database.items():\n        dist = np.linalg.norm(encoding - database[name])\n        if dist<min_dist:\n            min_dist = dist\n            identity = name\n    if min_dist > 0.7:\n        print("Intruder,distance is " + str(min_dist))\n    else:\n        print("Welcome " +str(identity) +",distance is " + str(min_dist))\n    return min_dist,identity\n\ndef img_to_encoding(image_path, model):\n    img1 = cv2.imread(image_path, 1)\n    img = img1[...,::-1]\n\n    classifier = cv2.CascadeClassifier(\'haarcascade_frontalface_default.xml\')\n    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)\n    faces = classifier.detectMultiScale(gray, 1.3, 5)\n    print("Found {0} face(s)!".format(len(faces)))\n    plt.figure(1)\n    for (x, y, w, h) in faces:\n        cv2.rectangle(img1, (x-10, y-10), (x+w+10, y+h+10), (0, 255, 0), 2)\n        img=img[y

In [ ]:
model = load_model()

In [ ]:
def recog(predict_image,database,model):
    recognize_face(predict_image,database,model)

In [8]:
def verify(image_path, identity, database, model):
    
    encoding = img_to_encoding(image_path, model, False)
    min_dist = 1000
    for  pic in database:
        dist = np.linalg.norm(encoding - pic)
        if dist < min_dist:
            min_dist = dist
    print(identity + ' : ' +str(min_dist)+ ' ' + str(len(database)))
    
    if min_dist<THRESHOLD:
        door_open = True
    else:
        door_open = False
        
    return min_dist, door_open


In [ ]:
#show_image('images/fnameTrue.jpg','images/fnameTest.jpg')
#add_face_database("images/s.jpg","Sachin",model)
add_face("pranav")

In [ ]:
add_face_database("images/pranav/pranav_1.jpg","Pranav",model)

In [ ]:
show_image('images/pranav/pranav_1.jpg','images/pranav/pranav_2.jpg')

In [ ]:
img_to_encoding("images/pranav/pranav_1.jpg",model)

In [ ]:
recog("images/TC.jpg",database,model)

In [9]:
best_model_path =""
if(os.path.exists("bestmodel.txt")):
    with open('bestmodel.txt', 'r') as file:
        best_model_path = file.read()
    
with open("./path_dict.p", 'rb') as f:
    paths = pickle.load(f)
    
faces = []
for key in paths.keys():
    paths[key] = paths[key].replace("\\", "/")
    faces.append(key)
    
if(len(faces) == 0):
    print("No images found in database!!")
    print("Please add images to database")
    sys.exit()

In [10]:
if os.path.exists(best_model_path) and best_model_path !="":
    print("Trained model found")
    print("Loading custom trained model...")
    FRmodel = keras.models.load_model(best_model_path,custom_objects={'triplet_loss': triplet_loss})
    
else:
    print("Custom trained model not found, Loading original facenet model...")
    FRmodel = faceRecoModel(input_shape=(3, IMAGE_SIZE, IMAGE_SIZE))
    load_weights_from_FaceNet(FRmodel)

Custom trained model not found, Loading original facenet model...


AttributeError: 'tuple' object has no attribute 'layer'

In [ ]:
database = {}
for face in faces:
    database[face] = []

for face in faces:
    for img in os.listdir(paths[face]):
        database[face].append(img_to_encoding(os.path.join(paths[face],img), FRmodel))


In [ ]:
camera = cv2.VideoCapture(0)
fd = faceDetector('fd_models/haarcascade_frontalface_default.xml')

fourcc = cv2.VideoWriter_fourcc(*'XVID') #codec for video
out = cv2.VideoWriter('output.avi', fourcc, 20, (800, 600) )#Output object

while True:
    ret, frame = camera.read()
    frame = imutils.resize(frame, width = 800)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    print(frame.shape)
    faceRects = fd.detect(gray)
    for (x, y, w, h) in faceRects:
        roi = frame[y:y+h,x:x+w]
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        roi = cv2.resize(roi,(IMAGE_SIZE, IMAGE_SIZE))
        min_dist = 1000
        identity = ""
        detected  = False
        
        for face in range(len(faces)):
            person = faces[face]
            dist, detected = verify(roi, person, database[person], FRmodel)
            if detected == True and dist<min_dist:
                min_dist = dist
                identity = person
        if detected == True:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, identity, (x+ (w//2),y-2), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), lineType=cv2.LINE_AA)
            
    cv2.imshow('frame', frame)
    out.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
out.release()
cv2.destroyAllWindows()
